In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
import os
import random
from warnings import warn

from noderedpy.api import NodeRedApi

GET MQTT BROKER and Inlfux DB

In [ ]:
BASE_URL = "http://127.0.0.1:1880"

In [ ]:
api = NodeRedApi(BASE_URL)

In [ ]:
api.get_mqtt_brokers()
api.get_influxdbs()

In [ ]:
mqtt_id = api.mqtt_brokers[0]['id']

In [ ]:
influx_id = api.influxdbs[0]['id']

CREATE A COPY OF WG1NSBRK

In [ ]:
from noderedpy.nodes import *

In [ ]:
# new flow / tab
f = Flow("wg1sbrk_copy2x")

project_name = "wg1nsbrk"


###############################
# steckdosen line
###############################
measurement = "TAS_C80F2A"

steckdose = MqttinNode(mqttbroker_id=mqtt_id, name="SteckDose", flow=f, topic=f"{project_name}/{measurement}/SENSOR/#", qos=0)
steckdose.x = 150
steckdose.y = 150

# create change node
steckdose_change = ChangeNode(name=f"{measurement}", flow=f, to_payload="ENERGY")

# connect MQTT and change node
connect_nodes(steckdose, steckdose_change, pos="right")

# create debug node
debug_steckdose_change = DebugNode(name="debug 31", flow=f)

# connect change and debug 
connect_nodes(steckdose_change, debug_steckdose_change, pos="bottom-right")

# Influx Output Node
steckdose_influx = InfluxdbOutNode(influxnode=influx,
                                   name=f"{project_name}/{measurement}",
                                   flow=f,
                                   influx_bucket=project_name,
                                   influx_measurement=measurement)

# connect change and influx
connect_nodes(steckdose_change, steckdose_influx, pos="top-right")

###############################
# Komfortmeter line
###############################
mqtt_broker_tls = MqttbrokerNode(name="bem.eco",
                             broker="bem.eco",
                             port="44220",
                            usetls=True)
tunnel = MqttinNode(broker=mqtt_broker_tls, name="TUNNEL", flow=f, topic="wg1nsbrk/#", qos=2)
tunnel.x = 150
tunnel.y = 500

debug_node = DebugNode(name="debug 32", flow=f)

for measurement, pos in zip(["ATCcff9430", "ATCaa91c5", "ATC700cde"],["top-right", "right", "bottom-right"]):
    tunnel_change = ChangeNode(name=measurement, flow=f, to_payload=measurement)
    connect_nodes(tunnel, tunnel_change, pos=pos)
    
    switch_node = SwitchNode(name=f'switch {measurement}', flow=f)
    connect_nodes(tunnel_change, switch_node, node2_output=1, pos="right")
    connect_nodes(switch_node, debug_node, node1_output=1, pos="bottom-right")
    
    influx_nodes = InfluxdbOutNode(influxnode=influx,name=f"{project_name}/{measurement}", flow=f, influx_bucket=project_name, influx_measurement=measurement)
    connect_nodes(switch_node, influx_nodes, node1_output=1, pos="top-right")

